<a href="https://colab.research.google.com/github/sliscak/e2n-translator/blob/main/english2narsese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL PREREQ

In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
#!cp "/content/drive/My Drive/research/datasets/dataset.jsonl" "/content"
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install -U nlp
!pip install pytorch-lightning
!pip install tqdm

!rm e2n-translator -rf
!git clone https://github.com/sliscak/e2n-translator.git
!python e2n-translator/dataset/build.py

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/5QHw3EvXYtlbRINlddc0SJpaLluqX80irYisAkuSH7PxXWSkuCFgRro


IMPORT PACKAGES

In [2]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, Trainer, TrainingArguments, T5Config, AdamW #T5WithLMHeadModel
import torch
from torch.utils.data import Dataset, DataLoader
import json
import nlp
from pytorch_lightning.core.lightning import LightningModule
import pytorch_lightning as pl
from os import path
from tqdm import tqdm

In [3]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
PATH = "/content/drive/My Drive/research/models/t5-py"
tokenizer.add_tokens('<')
# model = T5ForConditionalGeneration.from_pretrained('t5-base')

1

In [4]:
# print(tokenizer)
# from pprint import pprint
# pprint(dir(tokenizer))
# print(tokenizer.add_tokens()
# tokenizer.add_tokens('<')

In [5]:
# print(tokenizer.special_tokens_map)
# print(tokenizer.post_processor)

In [6]:
 class PythonDataset(Dataset):
    """Python dataset."""
    """Either preprocess or process data on the fly"""

    def __init__(self, file_path="/content/dataset.txt"):
      self.data = []
      with open(file_path, 'r') as fd:
        pair = []
        for line in tqdm(fd.readlines(), desc='Dataset Loading'):
          if len(line) > 3:
              pair.append(line)
              if len(pair) >= 2:
                self.data.append(pair)
                pair = [] 
        # else:
        #   for line in tqdm(fd.readlines(), desc='Dataset Loading'):
        #     # TODO: remove this part
        #     if len(line) > 3:
        #       self.data.append(line)

    def process(self, pair):
        with torch.no_grad():
          text, narsese = pair
          # narsese = narsese.replace('<', '(')
          # narsese = narsese.replace('>', ')')
          pattern = f'translate TEXT to NARSESE: "{text}"'

          input_ids = tokenizer.encode(pattern, padding=True, truncation=True, return_tensors='pt')
          labels = tokenizer.encode(narsese, padding=True, truncation=True, return_tensors='pt')
          sample = {
              "input_ids": input_ids,
              "labels": labels,
          }
          return sample

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        # print(self.data[idx])
        # processing on the fly
        return self.process(self.data[idx])

dataset = PythonDataset()
dataset[0]

Dataset Loading: 100%|██████████| 62/62 [00:00<00:00, 19852.42it/s]


{'input_ids': tensor([[13959,     3,  3463,     4,   382,    12,   445, 25210,  3205,   427,
             10,    96,   667,   210,    40,    19,     3,     9,   686,    13,
           2586,     5,    96,     1]]),
 'labels': tensor([[32100,     3,  2381,    40,     3,    18, 13114,  2586,  3155,     5,
              1]])}

In [15]:
class Pavian(LightningModule):
  def __init__(self): # reset=False
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained('t5-base')
    self.model.train()
    # self.save_hyperparameters()

  def forward(self, kwargs):
    return self.model(**kwargs)

  def configure_optimizers(self):
        optimizer = AdamW(model.parameters(), lr=1e-4)
        return optimizer

  def training_step(self, batch, batch_idx):
        for key in batch.keys():
          batch[key] = batch[key][0]
        outputs = self(batch)
        loss = outputs[0]
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}


  # def validation_step(self, batch, batch_idx):
  #       x, y = batch
  #       y_hat = self(x)
  #       val_loss = F.cross_entropy(y_hat, y)
  #       return val_loss
model = Pavian()

In [16]:
config = {
    'gpus': 1, # uncomment to use GPU
    'default_root_dir': '/content/drive/My Drive/research/checkpoints',
    'checkpoint_callback': False,
    'max_epochs': 50,
    # 'precision':16,
}



train_loader = DataLoader(PythonDataset(), batch_size=1, shuffle=True)

trainer = pl.Trainer(**config)
trainer.fit(model, train_loader)

Dataset Loading: 100%|██████████| 62/62 [00:00<00:00, 66508.15it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


1

In [17]:
# PATH = "/content/drive/My Drive/research/models/t5-py"
# torch.save(model.state_dict(), PATH)

In [22]:
with torch.no_grad():
    narsese = '<goat --> animal>.'
    pattern = f'translate TEXT to NARSESE: "Goat is a type of animal."'
    input_ids = tokenizer.encode(pattern, return_tensors='pt')
    outputs = model.model.generate(input_ids)
    dec = tokenizer.decode(outputs[0])
    print(f'Input:\n\t{pattern}\nOutput:\n\t{dec}\n')
    print(f'Ground Truth:\n\t{narsese}')


Input:
	translate TEXT to NARSESE: "Goat is a type of animal."
Output:
	 < goat --> animal>.

Ground Truth:
	<cat --> animal>.
